In [37]:

# Config (update these variables based on your GCP project and naming conventions)

GCP_PROJECT_ID           = 'GCPProject1'        # Google Cloud Project ID
BQ_DATASET               = 'OrderData'      # Name of BigQuery Dataset to create (if it does not exist)
BQ_MODEL_NAME            = 'shipDateDeterminationModel' # Name of BigQuery model that will be trained
GCS_LOCATION             = 'us-central1'    # Google Cloud Storage Location
GCS_BUCKET               = 'gs://aiplatformmodelregistry/ShipDateModel/'
VERTEX_LOCATION          = 'us-central1'
VERTEX_MODEL_NAME        = 'shipDateDeterminationModel'
VERTEX_SERVING_CONTAINER = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-1:latest' # https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

In [ ]:
!pip install google-cloud-aiplatform==1.0.1

In [ ]:
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [ ]:
# Initialize BQ client
try:
    bq_client = bigquery.Client()
except Exception as e: 
    print('[ EXCEPTION ] {}'.format(e))

QUERY = f'''
CREATE OR REPLACE MODEL
  `{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_MODEL_NAME}` OPTIONS(model_type='AUTOML_REGRESSOR',
               input_label_cols=['FULFILLMENT_DATE_IN_HOURS'],
               budget_hours=5.0,
               OPTIMIZATION_OBJECTIVE='MINIMIZE_RMSE')
AS
SELECT * 
FROM `ma-rd-devils-sren.OrderData.ProcessedOrderDetails`
'''
query_job = bq_client.query(QUERY)
rows = query_job.result()

In [ ]:
# Export BQML Model to the newly created Cloud Storage Bucket
bq_client = bigquery.Client()
QUERY = f'''
EXPORT MODEL `{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_MODEL_NAME}`
OPTIONS(URI = '{GCS_BUCKET}')
'''
print(QUERY)
query_job = bq_client.query(QUERY)
rows = query_job.result()

In [38]:
# Pre-built Model Serving Containers:
# https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

aiplatform.init(project=GCP_PROJECT_ID, location=VERTEX_LOCATION)
    
model = aiplatform.Model.upload(
    display_name=VERTEX_MODEL_NAME,
    artifact_uri=f'gs://aiplatformmodelregistry/ShipDateModel/',
    serving_container_image_uri=VERTEX_SERVING_CONTAINER
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/1018321702381/locations/us-central1/models/8344615955657129984/operations/1628306626869985280
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/1018321702381/locations/us-central1/models/8344615955657129984
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/1018321702381/locations/us-central1/models/8344615955657129984')


In [39]:
# Deploy Vertex Model as an Endpoint

endpoint = model.deploy(
    deployed_model_display_name=f'''{VERTEX_MODEL_NAME}-endpoint''',
    machine_type="n1-highcpu-4",
    min_replica_count=1,
    max_replica_count=3
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/1018321702381/locations/us-central1/endpoints/4373329489711595520/operations/2562803549549363200
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/1018321702381/locations/us-central1/endpoints/4373329489711595520
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/1018321702381/locations/us-central1/endpoints/4373329489711595520')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/1018321702381/locations/us-central1/endpoints/4373329489711595520
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/1018321702381/locations/us-central1/endpoints/4373329489711595520/operations/6239992645297373184


FailedPrecondition: 400 Error: model server never became ready. Please validate that your model file or container configuration are valid. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=1018321702381&resource=aiplatform.googleapis.com%252FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%224373329489711595520%22%0Aresource.labels.location%3D%22us-central1%22.